In [7]:
import requests
import os

os.environ["no_proxy"] = "localhost,172.16.87.75"

# API endpoint
url = "http://localhost:8000/lip_sync"



# Prepare the data
files = {
    'video': ('sun.mp4', open('data/video/sun.mp4', 'rb'), 'video/mp4'),
    'audio': ('sun.wav', open('data/audio/sun.wav', 'rb'), 'audio/wav'),
}

data = {
    'avatar_id': 'avator_1',
    'fps': 25,
    'batch_size': 4
}

# Send the POST request
response = requests.post(url, files=files, data=data, stream=True)

# Check if the request was successful
if response.status_code == 200:
    # The response is a stream of JPEG images
    # You can process them as they come or save them to a file
    with open('output_video.mp4', 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print("Lip-synced video saved as output_video.mp4")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

Error: 500
Internal Server Error
